In [67]:
import pandas as pd

Import all the files

In [68]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [69]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [70]:
visits_cart = pd.merge(visits,cart,how='left')

Step 3: How long is `visits_cart`?

In [71]:
visits_cart.info()
visits_cart.describe(datetime_is_numeric=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     2000 non-null   object        
 1   visit_time  2000 non-null   datetime64[ns]
 2   cart_time   348 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 62.5+ KB


,visit_time,cart_time
count,2000,348
mean,2017-06-17 07:51:59.490000128,2017-06-15 03:22:43.448275712
min,2017-01-01 03:47:00,2017-01-01 03:58:00
25%,2017-03-21 01:40:30,2017-03-18 05:15:30
50%,2017-06-19 13:39:30,2017-06-11 09:35:00
75%,2017-09-12 12:38:30,2017-09-06 06:56:00
max,2017-11-27 09:40:00,2017-11-27 08:54:00


Step 4: How many timestamps are null for `cart_time`?

In [72]:
total_visits = visits_cart.visit_time.count()
null_cart = visits_cart.cart_time.isna().sum()
print(total_visits)
print(null_cart)

2000
1652


Step 5: What percentage only visited?

In [73]:
only_visited = null_cart/total_visits * 100
print(f'{only_visited}%')

82.6%


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [74]:
from IPython.display import display

visits_cart_checkout = pd.merge(visits_cart,checkout,how='left')
visits_cart_checkout.info()
display(visits_cart_checkout.describe(datetime_is_numeric=True))

in_cart = visits_cart_checkout.cart_time.count()
checked = visits_cart_checkout.checkout_time.count()
cart_but_no_check = ((in_cart - checked)/in_cart) * 100
print(f'{round(cart_but_no_check,2)}%')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        2000 non-null   object        
 1   visit_time     2000 non-null   datetime64[ns]
 2   cart_time      348 non-null    datetime64[ns]
 3   checkout_time  226 non-null    datetime64[ns]
dtypes: datetime64[ns](3), object(1)
memory usage: 78.1+ KB


,visit_time,cart_time,checkout_time
count,2000,348,226
mean,2017-06-17 07:51:59.490000128,2017-06-15 03:22:43.448275712,2017-06-14 12:31:25.221239040
min,2017-01-01 03:47:00,2017-01-01 03:58:00,2017-01-04 05:32:00
25%,2017-03-21 01:40:30,2017-03-18 05:15:30,2017-03-20 05:38:30
50%,2017-06-19 13:39:30,2017-06-11 09:35:00,2017-06-09 20:32:00
75%,2017-09-12 12:38:30,2017-09-06 06:56:00,2017-09-05 04:33:15
max,2017-11-27 09:40:00,2017-11-27 08:54:00,2017-11-27 09:18:00


35.06%


Step 7: Merge it all together

In [104]:
print(purchase.info())
duplicates = purchase[purchase.duplicated(subset=['user_id'], keep=False)]
print(duplicates)
print(purchase['user_id'].duplicated().sum())
purchase = purchase.drop_duplicates(subset=['user_id'])
visits_cart_checkout_purchase = pd.merge(visits_cart_checkout,purchase,how='left')
visits_cart_checkout_purchase.info()
display(visits_cart_checkout_purchase.describe(datetime_is_numeric=True))



<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 251
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        144 non-null    object        
 1   purchase_time  144 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 3.4+ KB
None
Empty DataFrame
Columns: [user_id, purchase_time]
Index: []
0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        2000 non-null   object        
 1   visit_time     2000 non-null   datetime64[ns]
 2   cart_time      348 non-null    datetime64[ns]
 3   checkout_time  226 non-null    datetime64[ns]
 4   purchase_time  144 non-null    datetime64[ns]
dtypes: datetime64[ns](4), object(1)
memory usage: 93.8+ KB


,visit_time,cart_time,checkout_time,purchase_time
count,2000,348,226,144
mean,2017-06-17 07:51:59.490000128,2017-06-15 03:22:43.448275712,2017-06-14 12:31:25.221239040,2017-06-13 15:11:40.416666624
min,2017-01-01 03:47:00,2017-01-01 03:58:00,2017-01-04 05:32:00,2017-01-05 02:59:00
25%,2017-03-21 01:40:30,2017-03-18 05:15:30,2017-03-20 05:38:30,2017-03-25 06:57:45
50%,2017-06-19 13:39:30,2017-06-11 09:35:00,2017-06-09 20:32:00,2017-06-02 23:53:30
75%,2017-09-12 12:38:30,2017-09-06 06:56:00,2017-09-05 04:33:15,2017-09-04 06:44:15
max,2017-11-27 09:40:00,2017-11-27 08:54:00,2017-11-27 09:18:00,2017-11-24 05:25:00


Step 8: % of users who got to checkout but did not purchase

In [105]:
checked = visits_cart_checkout_purchase.checkout_time.count()
bought = visits_cart_checkout_purchase.purchase_time.count()
checked_but_no_bought = ((checked - bought)/checked) * 100
print(f'{round(checked_but_no_bought,2)}%')


36.28%


Step 9: check each part of the funnel, let's print all 3 of them again

In [106]:
print(f'percentage of users who visited the website but didn\'t start a cart: {only_visited}%')
print(f'percentage of users who started a cart but didn\'t check out: {round(cart_but_no_check,2)}%')
print(f'percentage of users who started a cart and checked out, but didn\'t complete purchase: {round(checked_but_no_bought,2)}%')

percentage of users who visited the website but didn't start a cart: 82.6%
percentage of users who started a cart but didn't check out: 35.06%
percentage of users who started a cart and checked out, but didn't complete purchase: 36.28%


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [107]:
all_data = visits_cart_checkout_purchase
all_data['time_spent'] = all_data.purchase_time - all_data.visit_time

Step 11: examine the results

In [108]:
time_spent_purchase = all_data[all_data.time_spent.notna()]
print(time_spent_purchase.time_spent.dt.total_seconds() / 3600)

1       0.733333
14      0.633333
48      0.683333
64      1.000000
69      0.566667
          ...   
1971    0.683333
1972    0.333333
1977    0.500000
1987    0.766667
1991    0.533333
Name: time_spent, Length: 144, dtype: float64


Step 12: average time to purchase

In [109]:
average_time = time_spent_purchase.time_spent.mean()
print(average_time)

0 days 00:42:33.333333333


In [110]:
print(all_data[['purchase_time', 'visit_time']].head(10))

        purchase_time          visit_time
0                 NaT 2017-04-07 15:14:00
1 2017-01-26 15:08:00 2017-01-26 14:24:00
2                 NaT 2017-08-20 08:23:00
3                 NaT 2017-11-04 18:15:00
4                 NaT 2017-02-27 11:25:00
5                 NaT 2017-06-03 20:05:00
6                 NaT 2017-03-18 23:09:00
7                 NaT 2017-01-10 12:09:00
8                 NaT 2017-06-27 19:21:00
9                 NaT 2017-09-24 06:43:00
